In [18]:
from keras import applications
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import keras
import os
from IPython.display import display
import PIL
from PIL import Image
from PIL.Image import core as _imaging
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix

from keras.applications.inception_v3 import preprocess_input
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
import tensorflow as tf
sess =tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [7]:
images_dataset_dir = "images"
ground_truth_dir = "annotations"
test_file = open('annotations/test.txt')
test_names_list = test_file.readlines()
train_file = open('annotations/trainval.txt')
train_names_list = train_file.readlines()

test_class_labels = [ name.split()[1] for name in test_names_list]
test_class_names = [ name.split()[0] for name in test_names_list]
test_name_label_dict = dict(zip(test_class_names,test_class_labels))

train_class_labels = [ name.split()[1] for name in train_names_list]
train_class_names = [ name.split()[0] for name in train_names_list]
train_name_label_dict = dict(zip(train_class_names,train_class_labels))

In [24]:
test_class_labels = np.array(test_class_labels).astype(int)
num_classes = np.unique(test_class_labels).shape[0]

In [58]:
img_rows, img_cols = 150,150

train_labels = []
train_data = []
test_data = []
test_labels =[]

for imageName in sorted(os.listdir("images")):
    if (".mat" in imageName):
        print(".mat error for file: ",imageName)
        continue
    img = load_img("images/"+str(imageName),False,target_size=(img_rows,img_cols,3),interpolation='nearest')
    x = img_to_array(img)
    breed_name = imageName[:-4]
    
    if breed_name in train_class_names:
        train_data.append(x)
        train_labels.append(train_name_label_dict[breed_name])
        
    elif breed_name in test_class_names:
        test_data.append(x)
        test_labels.append(test_name_label_dict[breed_name])
    else:
        print("Breed not found. Invalid image:", imageName, breed_name)

.mat error for file:  Abyssinian_100.mat
.mat error for file:  Abyssinian_101.mat
.mat error for file:  Abyssinian_102.mat
Breed not found. Invalid image: Abyssinian_34.jpg Abyssinian_34
Breed not found. Invalid image: Abyssinian_82.jpg Abyssinian_82
Breed not found. Invalid image: Bombay_11.jpg Bombay_11
Breed not found. Invalid image: Bombay_189.jpg Bombay_189
Breed not found. Invalid image: Bombay_190.jpg Bombay_190
Breed not found. Invalid image: Bombay_192.jpg Bombay_192
Breed not found. Invalid image: Bombay_203.jpg Bombay_203
Breed not found. Invalid image: Bombay_206.jpg Bombay_206
Breed not found. Invalid image: Bombay_209.jpg Bombay_209
Breed not found. Invalid image: Bombay_210.jpg Bombay_210
Breed not found. Invalid image: Bombay_217.jpg Bombay_217
Breed not found. Invalid image: Bombay_22.jpg Bombay_22
Breed not found. Invalid image: Bombay_220.jpg Bombay_220
Breed not found. Invalid image: Bombay_32.jpg Bombay_32
Breed not found. Invalid image: Bombay_69.jpg Bombay_69
Bre

In [59]:
train_labels_np = np.array(train_labels)
test_labels_np = np.array(test_labels)
train_data_np = np.array(train_data)
test_data_np = np.array(test_data)

In [60]:
model_vgg16 = applications.VGG16(include_top=False, weights='imagenet')

model_inc = applications.inception_v3.InceptionV3(include_top=False,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=(150,150,3),
                                                  pooling=None, classes=1000)

In [61]:
#train_pre = preprocess_input_vgg(train_data_np)
train_inc = preprocess_input(train_data_np)
test_inc = preprocess_input(test_data_np)
#test_pre = preprocess_input_vgg(test_data_np)

feat_train_inc = model_inc.predict(train_inc)
X_train = feat_train_inc.reshape(feat_train_inc.shape[0], -1)
y_train = np.array(train_labels_np,copy=True)

feat_test_inc = model_inc.predict(test_inc)
X_test = feat_test_inc.reshape(feat_test_inc.shape[0], -1)
y_test = np.array(test_labels_np,copy = True)

In [62]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)
y_train[y_train == 37] = 0
y_test[y_test == 37] = 0

In [92]:
from sklearn.svm import LinearSVC
logit = LinearSVC().fit(X_train, y_train)
print(logit.score(X_train, y_train))
print(logit.score(X_test, y_test))
confusion_matrix(y_test, logit.predict(X_test))

1.0
0.7972199509403107


array([[89,  0,  0, ...,  0,  1,  1],
       [ 0, 65,  1, ...,  3,  0,  0],
       [ 0,  0, 73, ...,  0,  3,  0],
       ...,
       [ 1,  4,  1, ..., 84,  0,  0],
       [ 0,  1,  8, ...,  0, 46,  0],
       [ 0,  0,  0, ...,  0,  0, 88]])